In [1]:
import pandas as pd

from neo4j import GraphDatabase

import os
from dotenv import load_dotenv

from tqdm import tqdm

load_dotenv()

True

In [2]:
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

print(uri, username, password)

neo4j+s://6ea87e27.databases.neo4j.io neo4j 6hKhbbaT_Yh53PXZYO5ivFsQXkq4NtOg43Ds3qoP4LE


In [3]:
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    driver.verify_connectivity()

In [4]:
player_df = pd.read_csv('../data/csv_data/(after) Player information features.csv')
action_df = pd.read_csv('../data/csv_data/(after) Player actions features.csv')
social_df = pd.read_csv('../data/csv_data/(after) Social interaction diversity features.csv')
network_df = pd.read_csv('../data/csv_data/(after) Network measures features.csv')
group_df = pd.read_csv('../data/csv_data/(after) Group activities features.csv')

In [5]:
player_df = player_df.drop('Type', axis=1)
action_df = action_df.drop('Type', axis=1)
social_df = social_df.drop('Type', axis=1)
group_df = group_df.drop('Type', axis=1)
network_df = network_df.drop('Type', axis=1)

In [6]:
player_df.head()
action_df.head()
# social_df.head()
# group_df.head()
# network_df.head()

,Actor,A_Acc,collect_max_count,Sit_ratio,Sit_count,sit_count_per_day,Exp_get_ratio,Exp_get_count,exp_get_count_per_day,Item_get_ratio,...,Use_portal_count,Use_portal_count_per_day,Killed_bypc_count,Killed_bypc_count_per_day,Killed_bynpc_count,Killed_bynpc_count_per_day,Teleport_count,Teleport_count_per_day,Reborn_count,Reborn_count_per_day
0,1047,6482393,6,1.0430,1012,24.0952,15.5210,15060,358.5714,6.2847,...,2,0.0476,690,16.4286,172,4.0952,1051,25.0238,2,0.0476
1,1049,6275719,0,3.5570,141,8.8125,13.6226,540,33.7500,6.9122,...,2,0.1250,30,1.8750,16,1.0000,118,7.3750,0,0.0000
2,1120,6596993,3,4.9882,190,47.5000,15.6734,597,149.2500,13.5994,...,0,0.0000,0,0.0000,20,5.0000,37,9.2500,0,0.0000
3,1164,6670686,0,0.8581,30,3.3333,17.8776,625,69.4444,6.8650,...,0,0.0000,26,2.8889,6,0.6667,61,6.7778,0,0.0000
4,1184,4220231,4,4.2667,215,19.5455,18.0194,908,82.5455,8.0572,...,0,0.0000,6,0.5455,17,1.5455,54,4.9091,0,0.0000


In [7]:
def create_player_nodes_batch(tx, player_data_list):
    query = """
        UNWIND $player_list AS playerData
        CREATE (p:Player)
        SET p = playerData
    """
    tx.run(query, player_list=player_data_list)

def create_action_nodes_batch(tx, action_data_list):
    query = """
        UNWIND $action_list AS actionData
        CREATE (a:Action)
        SET a = actionData
    """
    tx.run(query, action_list=action_data_list)

def create_performed_relationships_batch(tx, action_data_list):
    query = """
        UNWIND $action_list AS actionData
        MATCH (p:Player {Actor: toInteger(actionData.Actor)}),
              (a:Action {collect_max_count: toInteger(actionData.collect_max_count),
            Sit_ratio: toFloat(actionData.Sit_ratio),
            Sit_count: toInteger(actionData.Sit_count),
            sit_count_per_day: toFloat(actionData.sit_count_per_day),
            Exp_get_ratio: toFloat(actionData.Exp_get_ratio),
            Exp_get_count: toInteger(actionData.Exp_get_count),
            exp_get_count_per_day: toFloat(actionData.exp_get_count_per_day),
            Item_get_ratio: toFloat(actionData.Item_get_ratio),
            Item_get_count: toInteger(actionData.Item_get_count),
            item_get_count_per_day: toFloat(actionData.item_get_count_per_day),
            Money_get_ratio: toFloat(actionData.Money_get_ratio),
            Money_get_count: toInteger(actionData.Money_get_count),
            money_get_count_per_day: toFloat(actionData.money_get_count_per_day),
            Abyss_get_ratio: toFloat(actionData.Abyss_get_ratio),
            Abyss_get_count: toInteger(actionData.Abyss_get_count),
            abyss_get_count_per_day: toFloat(actionData.abyss_get_count_per_day),
            Exp_repair_count: toInteger(actionData.Exp_repair_count),
            Exp_repair_count_per_day: toFloat(actionData.Exp_repair_count_per_day),
            Use_portal_count: toInteger(actionData.Use_portal_count),
            Use_portal_count_per_day: toFloat(actionData.Use_portal_count_per_day),
            Killed_bypc_count: toInteger(actionData.Killed_bypc_count),
            Killed_bypc_count_per_day: toFloat(actionData.Killed_bypc_count_per_day),
            Killed_bynpc_count: toInteger(actionData.Killed_bynpc_count),
            Killed_bynpc_count_per_day: toFloat(actionData.Killed_bynpc_count_per_day),
            Teleport_count: toInteger(actionData.Teleport_count),
            Teleport_count_per_day: toFloat(actionData.Teleport_count_per_day),
            Reborn_count: toInteger(actionData.Reborn_count),
            Reborn_count_per_day: toFloat(actionData.Reborn_count_per_day)
        })
        CREATE (p)-[:PERFORMED]->(a)
    """
    tx.run(query, action_list=action_data_list)

def create_social_relationships_batch(tx, social_data_list):
    query = """
        UNWIND $social_list AS socialData
        MATCH (p:Player {Actor: toInteger(socialData.Actor)})
        SET p.Social_diversity = toFloat(socialData.Social_diversity)
    """
    tx.run(query, social_list=social_data_list)

def create_group_relationships_batch(tx, group_data_list):
    query = """
        UNWIND $group_list AS groupData
        MATCH (p:Player {Actor: toInteger(groupData.Actor)})
        SET p.Avg_PartyTime = toFloat(groupData.Avg_PartyTime),
            p.GuildAct_count = toInteger(groupData.GuildAct_count),
            p.GuildJoin_count = toInteger(groupData.GuildJoin_count)
    """
    tx.run(query, group_list=group_data_list)

def create_network_properties_batch(tx, network_data_list):
    query = """
        UNWIND $network_list AS networkData
        MATCH (p:Player {Actor: toInteger(networkData.Actor)})
        SET p += networkData
    """
    # Ensure properties like 'Actor' and 'A_Acc' are not included in networkData
    for data in network_data_list:
        data.pop('Actor', None)
        data.pop('A_Acc', None)

    tx.run(query, network_list=network_data_list)

In [8]:
batch_size = 1000  # Adjust batch size as needed

# Player Nodes
print("Preparing Player Data...")
player_data_list = [row.to_dict() for index, row in player_df.iterrows()]
print(player_data_list[0])

# Action Nodes and Relationships
print("Preparing Action Data...")
action_data_list = [row.to_dict() for index, row in action_df.iterrows()]
print(action_data_list[0])

# Social Relationships
print("Preparing Social Data...")
social_data_list = [row.to_dict() for index, row in social_df.iterrows()]
print(social_data_list[0])

# Group Relationships
print("Preparing Group Data...")
group_data_list = [row.to_dict() for index, row in group_df.iterrows()]
print(group_data_list[0])

# Network Properties
print("Preparing Network Data...")
network_data_list = [row.to_dict() for index, row in network_df.iterrows()]
print(network_data_list[0])


Preparing Player Data...
{'Actor': 1047.0, 'A_Acc': 6482393.0, 'Login_day_count': 46.0, 'Logout_day_count': 42.0, 'Playtime': 764520.0, 'playtime_per_day': 18202.8571, 'avg_money': 26576.5613, 'Login_count': 97.0, 'ip_count': 27.0, 'Max_level': 51.0}
Preparing Action Data...
{'Actor': 1047.0, 'A_Acc': 6482393.0, 'collect_max_count': 6.0, 'Sit_ratio': 1.043, 'Sit_count': 1012.0, 'sit_count_per_day': 24.0952, 'Exp_get_ratio': 15.521, 'Exp_get_count': 15060.0, 'exp_get_count_per_day': 358.5714, 'Item_get_ratio': 6.2847, 'Item_get_count': 6098.0, 'item_get_count_per_day': 145.1905, 'Money_get_ratio': 10.201, 'Money_get_count': 9898.0, 'money_get_count_per_day': 235.66666667, 'Abyss_get_ratio': 11.0636, 'Abyss_get_count': 10735.0, 'abyss_get_count_per_day': 255.5952, 'Exp_repair_count': 66.0, 'Exp_repair_count_per_day': 1.5714, 'Use_portal_count': 2.0, 'Use_portal_count_per_day': 0.0476, 'Killed_bypc_count': 690.0, 'Killed_bypc_count_per_day': 16.4286, 'Killed_bynpc_count': 172.0, 'Killed_b

In [9]:
# -----------------------------------------------------------------------------
# Neo4j Operations with Batching and Progress Bars
# -----------------------------------------------------------------------------

with driver.session() as session:
    # -------------------------------------------------------------------------
    # Create Player Nodes
    print("\nCreating Player Nodes...")
    with tqdm(total=len(player_data_list), desc="Player Nodes", unit="node") as pbar:
        for i in range(0, len(player_data_list), batch_size):
            batch = player_data_list[i:i + batch_size]
            session.execute_write(create_player_nodes_batch, batch)
            pbar.update(len(batch))
    print("Player Nodes Created.")

    # -------------------------------------------------------------------------
    # Create Action Nodes
    print("\nCreating Action Nodes...")
    with tqdm(total=len(action_data_list), desc="Action Nodes", unit="node") as pbar:
        for i in range(0, len(action_data_list), batch_size):
            batch = action_data_list[i:i + batch_size]
            session.execute_write(create_action_nodes_batch, batch)
            pbar.update(len(batch))
    print("Action Nodes Created.")
    
    print("\nCreating PERFORMED Relationships...")
    with tqdm(total=len(action_data_list), desc="PERFORMED Relationships", unit="rel") as pbar:
        for i in range(0, len(action_data_list), batch_size):
            batch = action_data_list[i:i + batch_size]
            session.execute_write(create_performed_relationships_batch, batch)
            pbar.update(len(batch))
    print("PERFORMED Relationships Created.")

    # -------------------------------------------------------------------------
    # Create Social Relationships
    print("\nCreating Social Relationships...")
    with tqdm(total=len(social_data_list), desc="Social Relationships", unit="rel") as pbar:
        for i in range(0, len(social_data_list), batch_size):
            batch = social_data_list[i:i + batch_size]
            session.execute_write(create_social_relationships_batch, batch)
            pbar.update(len(batch))
    print("Social Relationships Created.")

    # -------------------------------------------------------------------------
    # Create Group Relationships
    print("\nCreating Group Relationships...")
    with tqdm(total=len(group_data_list), desc="Group Relationships", unit="rel") as pbar:
        for i in range(0, len(group_data_list), batch_size):
            batch = group_data_list[i:i + batch_size]
            session.execute_write(create_group_relationships_batch, batch)
            pbar.update(len(batch))
    print("Group Relationships Created.")

    # -------------------------------------------------------------------------
    # Create Network Properties
    print("\nCreating Network Properties...")
    total_network_updates = len(network_data_list)
    with tqdm(total=total_network_updates, desc="Network Properties", unit="property") as pbar:
        for i in range(0, total_network_updates, batch_size):
            batch = network_data_list[i:i + batch_size]
            session.execute_write(create_network_properties_batch, batch)
            pbar.update(len(batch))
    print("Network Properties Created.")

print("\nKnowledge Graph Creation Complete!")

/var/folders/sy/4s0tvv155s961r1rz4db2sfw0000gn/T/ipykernel_45767/920335287.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:



Creating Player Nodes...


Player Nodes: 100%|██████████| 49739/49739 [00:19<00:00, 2593.87node/s]


Player Nodes Created.

Creating Action Nodes...


Action Nodes: 100%|██████████| 49739/49739 [00:32<00:00, 1544.39node/s]


...

Social Relationships Created.

Creating Group Relationships...


Group Relationships:  18%|█▊        | 9000/49739 [07:33<16:40, 40.74rel/s][#ECE6]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-6ea87e27-fa4a.production-orch-0695.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): TimeoutError('The read operation timed out')
Transaction failed and will be retried in 1.077783119892235s (Failed to read from defunct connection IPv4Address(('si-6ea87e27-fa4a.production-orch-0695.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))))
Group Relationships:  36%|███▌      | 18000/49739 [12:20<14:56, 35.39rel/s]  [#EFFB]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-6ea87e27-fa4a.production-orch-0695.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): TimeoutError('The read operation timed out')
Transaction failed and will be retried in 1.178647489985142s (Failed to read from defunct connection IPv4Address(('si-6ea87e27-fa4a.production-orch-0695.neo4j.

Group Relationships Created.

Creating Network Properties...


Network Properties: 100%|██████████| 49739/49739 [18:53<00:00, 43.88property/s]

Network Properties Created.

Knowledge Graph Creation Complete!


In [10]:
# -------------------------------------------------------------------------
# Create PERFORMED Relationships
